In [5]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [10]:

chrome_options = Options()
b = webdriver.Chrome()
b.get('https://weibo.com/')
time.sleep(10)
user = b.find_element_by_id('loginname')
pwd = b.find_element_by_name('password')
user.send_keys('zhaosichen4473@sina.cn')#account
pwd.send_keys('z1x91997zsc')#password
pwd.send_keys(Keys.ENTER)

In [12]:
cookie = b.get_cookies()
cookie
str1 = ""
for i in cookie:
    str1 += i["name"]+"="+i["value"]+"; "
    
str1

'webim_unReadCount=%7B%22time%22%3A1626805671670%2C%22dm_pub_total%22%3A0%2C%22chat_group_client%22%3A0%2C%22chat_group_notice%22%3A0%2C%22allcountNum%22%3A0%2C%22msgbox%22%3A0%7D; wvr=6; SSOLoginState=1626805649; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WhzpcRh1HdZrH1Ebu_UTb1v5JpX5KzhUgL.FozNe02E1K271hB2dJLoI7Uiqg4.wrSo; ALF=1658341647; SUB=_2A25N82XADeRhGeRJ6FMT-S_MwziIHXVuidAIrDV8PUNbmtAfLRTukW9NUrrjTi8A65KI_Qkl3hWlQy3fCsDlOkpX; wb_view_log_2731291084=1792*11202; wb_view_log=1792*11202; _s_tentry=-; Apache=2948407456706.6187.1626805564525; SINAGLOBAL=2948407456706.6187.1626805564525; __gads=ID=bcb1803f84e9eb00:T=1626805564:S=ALNI_MY6GfWQQ5RBx2w9W6XNbqiST1wT9A; _gid=GA1.2.368730099.1626805560; cross_origin_proto=SSL; _ga=GA1.2.1139622.1626805560; WBStorage=2ceabba76d81138d|undefined; ULV=1626805564530:1:1:1:2948407456706.6187.1626805564525:; login_sid_t=6c955ef550e235d5b97f76e93f3697f1; '

In [ ]:
# https://s.weibo.com/realtime?q=%23%E6%B2%B3%E5%8D%97%E6%9A%B4%E9%9B%A8%E4%BA%92%E5%8A%A9%23&rd=realtime&tw=realtime&Refer=weibo_realtime&page=1

In [89]:
def get_content(page):
    base_url = 'https://s.weibo.com/realtime?q=%23%E6%B2%B3%E5%8D%97%E6%9A%B4%E9%9B%A8%E4%BA%92%E5%8A%A9%23&rd=realtime&tw=realtime&Refer=weibo_realtime&page='
    url = base_url + str(page)
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Connection': 'keep-alive',
        'Cookie': str1,
        'Host': 's.weibo.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'
    }
    try:
        r = requests.get(url,headers = headers)
        soup = BeautifulSoup(r.text, "html.parser")
        return soup
    except:
        print('出错')



In [90]:
def regular(find_title):
    name = re.findall('.*?【姓名】(.*?)<', find_title)
    if len(name) != 0:
        name = ''.join(name)
    else:
        name = ""

    age = re.findall('.*?【年龄】(.*?)<', find_title)
    if len(age) != 0:
        age = ''.join(age)
    else:
        age = ""

    city = re.findall('.*?【所在城市】(.*?)<', find_title)
    if len(city) != 0:
        city = ''.join(city)
    else:
        city = ""
    village = re.findall('.*?【所在小区、社区】(.*?)<', find_title)
    if len(village) != 0:
        village = ''.join(village)
    else:
        village = ""
    ill_time = re.findall('.*?【患病时间】(.*?)<', find_title)
    if len(ill_time) != 0:
        ill_time = ''.join(ill_time)
    else:
        ill_time = ""
    describe = re.findall('.*?【病情描述】(.*?)<', find_title)
    if len(describe) != 0:
        describe = ''.join(describe)
    else:
        describe = ""
    phone = re.findall('.*?【联系方式】(.*?)<', find_title)
    if len(phone) != 0:
        phone = ''.join(phone)
    else:
        phone = ""
    other_way = re.findall('.*?【其他紧急联系人】(.*?)<', find_title)
    if len(other_way) != 0:
        other_way = ''.join(other_way)
    else:
        other_way = ""
    return name, age, city, village, ill_time, describe, phone, other_way


In [92]:
dic_all = {}
n = 0
for i in range(51):
    soup = get_content(i)
    for div in soup.find_all('div',{'class':"card"}):
        dic = {}
        for content in div.find_all('p',{'node-type':"feed_list_content"}):
            if '展开全文' in content.text:
                for content in div.find_all('p',{'node-type':"feed_list_content_full"}):  
                    dic['内容'] = re.sub('[\s+]','',content.text).strip('收起全文d')
#                     dic['内容'] = regular(re.sub('[\s+]','',content.text).strip('收起全文d'))
            else:
                dic['内容'] = re.sub('[\s+]','',content.text)
#                 dic['内容'] = regular(re.sub('[\s+]','',content.text))
        for time in div.find_all('p',{'class':"from"}):
            dic['时间和来源'] = re.sub('[\s+]','',time.text)
        dic_all[n] = dic
        n = n + 1
        print('已爬取{}条记录'.format(n))

已爬取1条记录
已爬取2条记录
已爬取3条记录
已爬取4条记录
已爬取5条记录
已爬取6条记录
已爬取7条记录
已爬取8条记录
已爬取9条记录
已爬取10条记录
已爬取11条记录
已爬取12条记录
已爬取13条记录
已爬取14条记录
已爬取15条记录
已爬取16条记录
已爬取17条记录
已爬取18条记录
已爬取19条记录
已爬取20条记录
已爬取21条记录
已爬取22条记录
已爬取23条记录
已爬取24条记录
已爬取25条记录
已爬取26条记录
已爬取27条记录
已爬取28条记录
已爬取29条记录
已爬取30条记录
已爬取31条记录
已爬取32条记录
已爬取33条记录
已爬取34条记录
已爬取35条记录
已爬取36条记录
已爬取37条记录
已爬取38条记录
已爬取39条记录
已爬取40条记录
已爬取41条记录
已爬取42条记录
已爬取43条记录
已爬取44条记录
已爬取45条记录
已爬取46条记录
已爬取47条记录
已爬取48条记录
已爬取49条记录
已爬取50条记录
已爬取51条记录
已爬取52条记录
已爬取53条记录
已爬取54条记录
已爬取55条记录
已爬取56条记录
已爬取57条记录
已爬取58条记录
已爬取59条记录
已爬取60条记录
已爬取61条记录
已爬取62条记录
已爬取63条记录
已爬取64条记录
已爬取65条记录
已爬取66条记录
已爬取67条记录
已爬取68条记录
已爬取69条记录
已爬取70条记录
已爬取71条记录
已爬取72条记录
已爬取73条记录
已爬取74条记录
已爬取75条记录
已爬取76条记录
已爬取77条记录
已爬取78条记录
已爬取79条记录
已爬取80条记录
已爬取81条记录
已爬取82条记录
已爬取83条记录
已爬取84条记录
已爬取85条记录
已爬取86条记录
已爬取87条记录
已爬取88条记录
已爬取89条记录
已爬取90条记录
已爬取91条记录
已爬取92条记录
已爬取93条记录
已爬取94条记录
已爬取95条记录
已爬取96条记录
已爬取97条记录
已爬取98条记录
已爬取99条记录
已爬取100条记录
已爬取101条记录
已爬取102条记录
已爬取103条记录
已爬取104条记录
已爬取105条记录
已爬取106条记录
已爬取107条记录
已爬取108条记录
已爬取109条记录
已爬取110条记录
已爬取111条记

已爬取844条记录
已爬取845条记录
已爬取846条记录
已爬取847条记录
已爬取848条记录
已爬取849条记录
已爬取850条记录
已爬取851条记录
已爬取852条记录
已爬取853条记录
已爬取854条记录
已爬取855条记录
已爬取856条记录
已爬取857条记录
已爬取858条记录
已爬取859条记录
已爬取860条记录
已爬取861条记录
已爬取862条记录
已爬取863条记录
已爬取864条记录
已爬取865条记录
已爬取866条记录
已爬取867条记录
已爬取868条记录
已爬取869条记录
已爬取870条记录
已爬取871条记录
已爬取872条记录
已爬取873条记录
已爬取874条记录
已爬取875条记录
已爬取876条记录
已爬取877条记录
已爬取878条记录
已爬取879条记录
已爬取880条记录
已爬取881条记录
已爬取882条记录
已爬取883条记录
已爬取884条记录
已爬取885条记录
已爬取886条记录
已爬取887条记录
已爬取888条记录
已爬取889条记录
已爬取890条记录
已爬取891条记录
已爬取892条记录
已爬取893条记录
已爬取894条记录
已爬取895条记录
已爬取896条记录
已爬取897条记录
已爬取898条记录
已爬取899条记录
已爬取900条记录
已爬取901条记录
已爬取902条记录
已爬取903条记录
已爬取904条记录
已爬取905条记录
已爬取906条记录
已爬取907条记录
已爬取908条记录
已爬取909条记录
已爬取910条记录
已爬取911条记录
已爬取912条记录
已爬取913条记录
已爬取914条记录
已爬取915条记录
已爬取916条记录
已爬取917条记录
已爬取918条记录
已爬取919条记录
已爬取920条记录
已爬取921条记录
已爬取922条记录
已爬取923条记录
已爬取924条记录
已爬取925条记录
已爬取926条记录
已爬取927条记录
已爬取928条记录
已爬取929条记录
已爬取930条记录
已爬取931条记录
已爬取932条记录
已爬取933条记录
已爬取934条记录
已爬取935条记录
已爬取936条记录
已爬取937条记录
已爬取938条记录
已爬取939条记录
已爬取940条记录
已爬取941条记录
已爬取942条记录
已爬取943条记录


In [93]:
pd.set_option('display.max_rows', None) # 展示所有行
pd.set_option('display.max_columns', None) # 展示所有列

In [94]:
data = pd.DataFrame(dic_all).T.dropna()

In [95]:
data

,内容,时间和来源
0,郑州市金水区已打电话核实需求内容：请求救援被救护人姓名：蔡能博被救护人电话：18695890...,5秒前来自王昊迷妹的Android
1,#河南暴雨互助#帮扩❗❗❗❗郑州陇海路第19高级中学！！！5个大人一个小孩求救🆘求救🆘求救🆘...,6秒前
2,现郑州源创运输有限公司，挖机三台，铲车一台，运输车辆50台。免费救援，需要的随时联系，电话：...,6秒前
3,最揪心的还是医院的情况，重症ICU和重症新生儿病房NICU没电只要几分钟需要机械呼吸支持的病...,6秒前来自vivoX274800万三摄
4,#河南暴雨互助#志愿群里发的。高铁东站附近能提供免费水和食物的去处。​,7秒前来自Android
5,郑州【求救‼️】郑州西三环东原阅城售楼部附近，情况从下午三点钟被困在车上，现在喝的水也快没...,7秒前来自荣耀7C人脸识别双摄
6,郑州我是帮转‼️#河南暴雨互助##郑州暴雨##郑州暴雨救援##郑州暴雨互助#求求大家关注一...,7秒前来自某被踩烂的Android
7,郑州#河南暴雨互助##河南暴雨#郑州帮转，最后百分之一的电了，京广路隧道水深最高两米，快...,7秒前来自OPPOA11x
8,永远无法抹去泄洪记忆的我，恳请各位密切关注#河南暴雨互助#信息，非常感谢！⚠️微博在线求助O...,8秒前来自HUAWEIP30
9,最新寻人启事#河南暴雨互助#穿着：黑色短裤！黑色书包！地址：东大街站下地铁7月20日下午6点...,8秒前


In [98]:
data.to_markdown()

'|      | 内容                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [96]:
data.to_excel(r'C:\Users\weibo.xls')

<ipython-input-96-772446906461>:1: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  data.to_excel(r'C:\Users\weibo.xls')
